In [1]:
!pip install pytorch_lightning
!pip install wandb
!pip install EMD-signal
!pip install ta
import statsmodels
if statsmodels.__version__ != "0.12.2":
  !pip install --upgrade statsmodels

     |████████████████████████████████| 916 kB 5.1 MB/s 
     |████████████████████████████████| 636 kB 43.1 MB/s 
     |████████████████████████████████| 829 kB 36.4 MB/s 
     |████████████████████████████████| 272 kB 50.0 MB/s 
     |████████████████████████████████| 118 kB 59.6 MB/s 
     |████████████████████████████████| 1.3 MB 41.7 MB/s 
     |████████████████████████████████| 294 kB 51.5 MB/s 
     |████████████████████████████████| 142 kB 53.2 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=45eb06f17b51c14a54fe017ec7619375f500111b4f2aab0e581430797494660a
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [1]:
!nvidia-smi

Thu Aug 19 06:24:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import os 
os.chdir('/content/gdrive/MyDrive/DL4Fin/multi_task_price_change_prediction/notebooks')

Mounted at /content/gdrive


In [3]:
import pytorch_lightning as pl

from torch.nn import functional as F
from torch import nn
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger

import wandb

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import torchmetrics
import pytorch_lightning as pl
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
from TimeSeriesLearningUtils import TimeSeriesDataset, CosineWarmupScheduler, get_data
from TransformerEncoderModel import TradePredictor
from LSTMModel import LSTM_based_classification_model

In [77]:
class LSTM_based_classification_model(pl.LightningModule):
    activations = {
        'relu': nn.ReLU(),
        'sigmoid': nn.Sigmoid(),
        'tanh': nn.Tanh(),
        'softmax': nn.Softmax()
        }
    
    def __init__(self,
                 train_dataset,
                 val_dataset,
                 test_dataset,
                 parameters,
                 learning_rate = 1e-3,
                 scheduler_step = 10,
                 scheduler_gamma = 0.1,
                 ):
        
        super().__init__()
      
        self.n_classes = parameters.n_classes
        self.currencies = parameters.currency_list
        self.num_tasks = len(self.currencies)
        self.window_size = parameters.window_size
        self.input_size = train_dataset.x.shape[-1]
        self.batch_size = parameters.batch_size
        self.lstm_hidden_sizes = parameters.lstm_hidden_sizes
        self.n_lstm_layers = len( self.lstm_hidden_sizes)
        self.bidirectional = parameters.bidirectional
        self.loss_weightening = parameters.loss_weightening
        self.dropout_after_each_lstm_layer = parameters.dropout_after_each_lstm_layer
        self.dropout_before_output_layer = parameters.dropout_before_output_layer
        
        if self.loss_weightening:
            loss_weights = []
            for i in range(self.num_tasks):
                train_labels = [int(train_dataset[n][self.currencies[i] +"_label"] )for n in range(len(train_dataset))]
                samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
                loss_weights.append((1 / samples_size) * sum(samples_size)/2)
            self.weights = loss_weights
        else:
            self.weights = None 

        self.lstm_blocks = nn.ModuleList()
        
        for i in range(self.n_lstm_layers):

            if i == 0:
              input_size = self.input_size 
            else:
              input_size = self.lstm_hidden_sizes[i-1]*2 if self.bidirectional else self.lstm_hidden_sizes[i-1]   
            
            lstm_layer = nn.LSTM(input_size = input_size, 
                                  num_layers=1, 
                                  batch_first=True, 
                                  hidden_size = self.lstm_hidden_sizes[i], 
                                  bidirectional = self.bidirectional)
            
            n_feature = self.lstm_hidden_sizes[i]*2 if self.bidirectional else self.lstm_hidden_sizes[i]   
            batch_norm = nn.BatchNorm2d(num_features=n_feature)
            lst = [('lstm', lstm_layer), ('batch_norm', batch_norm)]
  
            if self.dropout_after_each_lstm_layer:
                dropout = nn.Dropout(self.dropout_after_each_lstm_layer)
                lst.append(('dropout', dropout))
                
            module_dict = nn.ModuleDict(lst)
            
            self.lstm_blocks.append(module_dict)
            
        n_feature = self.lstm_hidden_sizes[-1]*2 if self.bidirectional else self.lstm_hidden_sizes[-1]
        
        self.linear1 =[nn.Linear(n_feature, n_feature//2)] * self.num_tasks
        self.linear1 = torch.nn.ModuleList(self.linear1)
        self.activation1 = nn.ReLU()

        if self.dropout_before_output_layer:
          self.dropout1 = nn.Dropout(self.dropout_before_output_layer)
        
        self.output_layers = [nn.Linear(n_feature//2, self.n_classes)] * self.num_tasks
        self.output_layers = torch.nn.ModuleList(self.output_layers)
        
        if self.weights != None:
            self.cross_entropy_loss = [nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) for weights in self.weights]
        else:
            self.cross_entropy_loss = [nn.CrossEntropyLoss() for _ in range(self.num_tasks)]
        
        self.cross_entropy_loss = torch.nn.ModuleList(self.cross_entropy_loss)
        
        self.f1_score = torchmetrics.classification.f_beta.F1(num_classes=self.n_classes, average="macro")
        self.accuracy_score = torchmetrics.classification.accuracy.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=self.batch_size, shuffle = True)
        self.val_dl = DataLoader(val_dataset, batch_size=self.batch_size)
        self.test_dl = DataLoader(test_dataset, batch_size=self.batch_size)
        
        self.learning_rate = learning_rate
        self.scheduler_step = scheduler_step
        self.scheduler_gamma = scheduler_gamma
        
    def forward(self, x, i):

        batch_size = x.size()[0]
 
        for i, block in enumerate(self.lstm_blocks):

            if i == 0:
              n_feature = self.input_size 
            else:
              n_feature = self.lstm_hidden_sizes[i-1]*2 if self.bidirectional else self.lstm_hidden_sizes[i-1]   
 
            x = x.view(batch_size, self.window_size, n_feature) #(batch, window_len, feature_size)
            x, _ = block['lstm'](x)
        
            if 'dropout' in block:
                x = block['dropout'](x)
           
            x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        
            x = block['batch_norm'](x.unsqueeze(0))
  
            if len(x.shape) == 4: #error handling
              x = x.squeeze() 

        n_feature = self.lstm_hidden_sizes[-1]*2 if self.bidirectional else self.lstm_hidden_sizes[-1]
        x = x.view(batch_size, self.window_size, n_feature)
        x = x[:, -1, :] # equivalent to return sequence = False on keras :)
      
        x = self.linear1[i](x)
        x = self.activation1(x)
        
        if self.dropout_before_output_layer:
            x = self.dropout1(x)
            
        output = self.output_layers[i](x)
        
        return output
    
    
    def training_step(self, batch, batch_nb):
        
        loss = (torch.tensor(0.0, device="cuda:0", requires_grad=True) + \
                torch.tensor(0.0, device="cuda:0", requires_grad=True)) 
        # araştırılabilir
        for i in range(self.num_tasks):
            x, y = batch[self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self.forward(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_train_acc', acc, on_epoch=True, prog_bar=True)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_train_f1', f1, on_epoch=True, prog_bar=True)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        
        return loss 
    
    def validation_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch[self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
 
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_val_acc', acc, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_val_f1', f1, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
    
    def test_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch[ self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self(x, i)
#             print(y, torch.max(output, dim=1)[1])
#             print(F.softmax(output)) # mantıken fark etmiyor
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_test_acc', acc, on_epoch=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_test_f1', f1, on_epoch=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('test_loss', loss, on_epoch=True, reduce_fx=torch.mean)

        
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(self.parameters(), lr= self.learning_rate)#AdamW does weight decay
#         scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
#                                                     step_size=self.scheduler_step, 
#                                                     gamma=self.scheduler_gamma)
        
        self.lr_scheduler = CosineWarmupScheduler(optimizer, 
                                                  warmup=self.train_dl.__len__() * 10, 
                                                  max_iters = 80 * self.train_dl.__len__())
        return [optimizer]#, [{"scheduler": scheduler}]
    
    def optimizer_step(self, *args, **kwargs):
        super().optimizer_step(*args, **kwargs)
        self.lr_scheduler.step() # Step per iteration
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

In [64]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 currency_list,
                 dataset_percentages,
                 window_size, 
                 **kwargs
                 ):
        self.currencies = currency_list
        self.n_currencies = len(self.currencies)
        self.x = torch.tensor(x[:self.n_currencies]).float()
        self.y = torch.tensor(y[:self.n_currencies]).long()
        self.seq_len = window_size
        self.data_use_type = data_use_type
        
        train_percentage,val_percentage,test_percentage = dataset_percentages
        self.train_size = int(len(self.x[0]) * train_percentage)
        self.val_size = int(len(self.x[0]) * val_percentage)
        self.test_size = len(self.x[0]) - self.train_size - self.val_size
        
        self.train_mean = [self.x[i][:self.train_size].mean() for i in range(self.n_currencies)]
        self.train_std = [self.x[i][:self.train_size].std() for i in range(self.n_currencies)]
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)

        elif self.data_use_type == "val":
            return self.val_size 
        else:
            return self.test_size
    
    def __getitem__(self, index):
        
        item = dict()
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        for i in range(self.n_currencies):
            window = self.x[i][index:index+self.seq_len]
            window = (window -self.train_mean[i]) / self.train_std[i]
            
            item[self.currencies[i] + "_window"] = window
            item[self.currencies[i] + "_label"]  = self.y[i][index+self.seq_len]

        return item

In [78]:
model_classes = {'lstm': LSTM_based_classification_model, 'transformer':TradePredictor}

In [80]:
data_setting = {
                "window_size": [25, 50, 75, 100, 125, 150],
                "ma_period": [7],
                "dataset_percentages": [[0.97, 0.007, 0.023], [0.8, 0.05, 0.15]],
                "data_frequency": ["1d"],
                "pred_frequency": ["D"],
                "neutral_quantile": [0.1, 0.25, 0.33],
                "n_classes": [2, 3],
                "currency_list": [['BTC', 'ETH', 'LTC', 'ADA', 'XRP']],
                "log_price":[True],
                "remove_trend": [True],
                "indicators": [False], 
                "imfs": [False],
                "ohlv": [False] #open-high-low-volume
                }

model_parameters = {}
model_parameters['lstm'] = {
                            "lstm_hidden_sizes": [[16], [32], [64], [128], [16, 16], [32, 32], [64, 64], [16, 32], [32, 64], [64, 128], [16, 32, 64, 128]], #lots of combinations...
                            "bidirectional": [True], 
                            "dropout_after_each_lstm_layer": [0, 0.25, 0.5],
                            "dropout_before_output_layer": [0, 0.25, 0.5],
                            "batch_size": [8, 16, 32, 64],
                            "loss_weightening": [False],
                            }
model_parameters['lstm'].update(data_setting)

model_parameters['transformer'] = {
                                    "batch_size": [8, 16, 32, 64],
                                    "loss_weightening": [False]
                                  }
model_parameters['transformer'].update(data_setting)

In [81]:
metric = {
  'name': 'val_loss',
  'goal': 'minimize'   
  }
sweep_configs = {}    
default_configs = {}
for model_name in model_classes:
    
    #only adding "values" keyword which is required for Bayesian search of wandb sweep
    parameters = {'model_name': {'values':[model_name]}}
    config_default = {}
    for key, value in model_parameters[model_name].items():
        if isinstance(value, dict):
          parameters[key] = {} 
          config_default[key] = {}
          for param_name, values in value.items():
              parameters[key][param_name] = {'values':values}
              config_default[key][param_name] = values[0]
        else:
          parameters[key] = {'values':value}
          config_default[key] = value


    sweep_config={'method': 'bayes',
                  'metric': metric,
                  'parameters': parameters}
    
    sweep_configs[model_name] = sweep_config
    default_configs[model_name] = config_default

In [47]:
def name_model(config):
    name =[]
    if len(config["currency_list"])  > 1:
        name.append("multi_task_" + "_".join(config["currency_list"]))
    else:
        name.append(config["currency_list"][0])
        
    if config["indicators"] or config["imfs"] or config ["ohlv"]:
        name.append("multi_variate")
    
    lstm = "stack_lstm" if len(config["lstm_hidden_sizes"]) > 1 else "lstm"
    name.append(lstm)
    
    classification = "multi_clf" if config["n_classes"] > 2 else "binary_clf"
    name.append(classification)
    
    return "_".join(name)

In [57]:
def train(config=None):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    with wandb.init(config=config, reinit=False) as run:

        config = run.config
        logger = WandbLogger()
        logger.log_hyperparams(config)
        
        X, y, features, dfs = get_data(**config)
        train_dataset, val_dataset, test_dataset = [TimeSeriesDataset(X, y, dtype, **config) for dtype in ['train', 'val', 'test']]
        model = model_classes[config.model_name](train_dataset, val_dataset, test_dataset, config)
        model.to(device)

        early_stop_callback = EarlyStopping(monitor='val_loss',
                                            min_delta=0.003,
                                            patience=20,
                                            verbose=True,
                                            mode='min')
        
        model_name = name_model(config) 
        
        checkpoint_callback = ModelCheckpoint(monitor='val_loss',
                                              dirpath='../output/',
                                              filename= model_name+'-{epoch:02d}-{val_loss:.2f}',
                                              save_top_k=1,
                                              mode='min')
  
        trainer = pl.Trainer(gpus=-1, 
                             max_epochs= 80,
                             logger = logger, 
                             callbacks=[early_stop_callback, checkpoint_callback])
        trainer.fit(model)
        trainer.test(ckpt_path = checkpoint_callback.best_model_path)


In [82]:
from pprint import pprint
for model_name in model_classes:
    if model_name == 'lstm':
      MODEL_NAME = model_name
      sweep_config = sweep_configs[model_name]
      pprint(sweep_config)
      sweep_id = wandb.sweep(sweep_config, entity='multi_task_price_prediction', project=model_name+'_tuning')
      wandb.agent(sweep_id, function=train)
      wandb.finish()

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'values': [8, 16, 32, 64]},
                'bidirectional': {'values': [True]},
                'currency_list': {'values': [['BTC',
                                              'ETH',
                                              'LTC',
                                              'ADA',
                                              'XRP']]},
                'data_frequency': {'values': ['1d']},
                'dataset_percentages': {'values': [[0.97, 0.007, 0.023],
                                                   [0.8, 0.05, 0.15]]},
                'dropout_after_each_lstm_layer': {'values': [0, 0.25, 0.5]},
                'dropout_before_output_layer': {'values': [0, 0.25, 0.5]},
                'imfs': {'values': [False]},
                'indicators': {'values': [False]},
                'log_price': {'values': [True]},
                'loss_weightening': {'values': [Fal

wandb: Agent Starting Run: h7myywbw with config:
wandb: 	batch_size: 8
wandb: 	bidirectional: True
wandb: 	currency_list: ['BTC', 'ETH', 'LTC', 'ADA', 'XRP']
wandb: 	data_frequency: 1d
wandb: 	dataset_percentages: [0.8, 0.05, 0.15]
wandb: 	dropout_after_each_lstm_layer: 0
wandb: 	dropout_before_output_layer: 0.5
wandb: 	imfs: False
wandb: 	indicators: False
wandb: 	log_price: True
wandb: 	loss_weightening: False
wandb: 	lstm_hidden_sizes: [16, 32]
wandb: 	ma_period: 7
wandb: 	model_name: lstm
wandb: 	n_classes: 2
wandb: 	neutral_quantile: 0.1
wandb: 	ohlv: False
wandb: 	pred_frequency: D
wandb: 	remove_trend: True
wandb: 	window_size: 75


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ../output/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type       | Params
--------------------------------------------------
0 | lstm_blocks        | ModuleList | 19.5 K
1 | linear1            | ModuleList | 2.1 K 
2 | activation1        | ReLU       | 0     
3 | dropout1           | Dropout    | 0     
4 | output_layers      | ModuleList | 66    
5 | cross_entropy_loss | ModuleList | 0     
6 | f1_score           | F1         | 0     
7 | accuracy_score     | Accuracy   | 0     
--------------------------------------------------
21.7 K    Trainable params
0         Non-trainable params
21.7 K    Total params
0.087     Total estim

Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

...

Metric val_loss improved. New best score: 0.704


Validating: 0it [00:00, ?it/s]

Metric val_loss improved by 0.004 >= min_delta = 0.003. New best score: 0.700
wandb: Ctrl + C detected. Stopping sweep.


BTC_train_acc_step,0.5
BTC_train_f1_step,0.5
ETH_train_acc_step,0.375
ETH_train_f1_step,0.36508
LTC_train_acc_step,0.25
LTC_train_f1_step,0.2
ADA_train_acc_step,0.625
ADA_train_f1_step,0.56364
XRP_train_acc_step,0.25
XRP_train_f1_step,0.2
train_loss_step,0.72598


BTC_train_acc_step,▁▁▁█▁▅
BTC_train_f1_step,▁▁▁█▃▆
ETH_train_acc_step,█▃▁▆▃▃
ETH_train_f1_step,█▃▁▇▁▃
LTC_train_acc_step,▃▆▆▁█▁
LTC_train_f1_step,▅█▄▁▆▁
ADA_train_acc_step,█▃▁▃▅▆
ADA_train_f1_step,█▃▁▃▅▆
XRP_train_acc_step,▆█▆▃▃▁
XRP_train_f1_step,▆█▇▂▄▁
train_loss_step,▁▂█▆▅▇


In [ ]:
!git push --set-upstream origin master

Counting objects: 12, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (12/12), done.
Writing objects: 100% (12/12), 49.41 KiB | 4.49 MiB/s, done.
Total 12 (delta 8), reused 0 (delta 0)
remote: Resolving deltas: 100% (8/8), completed with 4 local objects.
remote: 
remote: Create a pull request for 'tuning' on GitHub by visiting:
remote:      https://github.com/aysenurk/multi_task_price_change_prediction/pull/new/tuning
remote: 
To https://github.com/aysenurk/multi_task_price_change_prediction
 * [new branch]      tuning -> tuning
Branch 'tuning' set up to track remote branch 'tuning' from 'origin'.


In [84]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	10_fc_Tuning.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [83]:
!git config --global user.email "furkan.canturk@ozu.edu.tr"
!git config --global user.name "furkancanturk"